In [1]:
from charm.toolbox.pairinggroup import PairingGroup, G1,G2,GT, ZR, pair as e,extract_key
from hashlib import sha256, md5
from util1 import obj2bytes, read_word_as_bytes, bxor
import numpy as np  
import logging
import pickle
import socketserver
import sys
import import_ipynb
from Hash_Util import *
import copy

#权限机构服务器
class serverReqHandler(socketserver.BaseRequestHandler):
    def __init__(self, request, addr, server):
        super().__init__(request, addr, server)

    def handle(self):
        data1 = b""
        while True:
            packet = self.request.recv(4096)
            if packet is not None:
                data1 += packet
            if packet is None or len(packet) < 4096:
                break
        resp_tup = pickle.loads(data1)
        if(resp_tup[0] == 0):  # for setup
            data = self.server.Setup(resp_tup[1])
            logging.debug("Setup completed")
        elif(resp_tup[0] == 1):
            data = self.server.KeyGen(resp_tup[1])
            logging.debug("KeyGen completed")
        elif(resp_tup[0] == 2):
            data = self.server.UpdateKeyGen(resp_tup[1])
            logging.debug("UpdateKeyGen completed")
        elif(resp_tup[0] == 3):
            data = self.server.GetAllNeed()
            logging.debug("GetAllNeed completed")
        elif(resp_tup[0] == 4):
            data = self.server.SendRL_List()
            logging.debug("Send RL List completed")

        self.request.sendall(pickle.dumps(data))
        logging.debug('handled')
class TA_Handle(socketserver.TCPServer):
    def __init__(self, addr, handler_class=serverReqHandler) -> None:
        self.group = PairingGroup('MNT159')
        self.g = self.group.random(G1)
        self.g2 = self.group.random(G2)
        self.AVK = None
        self.APK = None
        self.alpha = None
        self.x_11 = None
        self.U = None       #全部属性
        self.U_Mapping = None      #数字排列与属性映射表
        self.x = None
        self.p = 625852803282871856053922297323874661378036491717
        self.lm =  b'\xfd' * 64
        self.RL = {}
        self.PK = None
        self.MSK = None
        self.h = None
        super().__init__(addr, handler_class)
    

    def Setup(self,Up):
        #系统初始化，生成公钥PK与主密钥MSK
        self.U_Mapping = Up[1]
        self.U = Up[0]
        self.x_11,self.x,self.alpha = self.group.random(ZR, 3)

        B_0 = self.g ** self.alpha 
        B_1 = self.g ** self.x_11
        self.AVK = {}
        self.APK = {}
        AVK_s = {}
        APK_s = {}
        for sita in self.U:
            v_sita = self.group.random(ZR)
            #对AVK 与 APK 中 Pairing.Element类型数据进行bytes化
            self.AVK[sita] = v_sita
            AVK_s[sita] = self.group.serialize(v_sita)
            self.APK[sita] = self.g ** v_sita
            APK_s[sita] = self.group.serialize(self.APK[sita])
            
        #对Pairing.Element类型数据进行bytes化
        x_s = self.group.serialize(self.x)
        x_11_s = self.group.serialize(self.x_11)
        g_s = self.group.serialize(self.g)
        g2_s = self.group.serialize(self.g2)
        B_0_s = self.group.serialize(B_0)
        B_1_s = self.group.serialize(B_1)
        self.h = H
        h_0 = H_0
        h_1 =H_1
        self.PK = [G1 , GT , self.p , self.g ,self.h,h_0,h_1,B_0,B_1,self.APK,self.g2]
        self.MSK = [self.x,self.x_11,self.AVK]
        PK_s =  [G1 , GT , self.p , g_s ,self.h,h_0,h_1,B_0_s,B_1_s,APK_s,g2_s]
        MSK_s = [x_s,x_11_s,AVK_s]
        return {'PK': PK_s, 'MSK' : MSK_s}
    def KeyGen(self,MS):
        #密钥获取操作，通过主密钥和属性S生成私钥SK
        msk = MS[0]
        S = MS[1]
        
        msk[0] = self.group.deserialize(msk[0])
        msk[1] = self.group.deserialize(msk[1])
        for key in msk[2].keys():
            msk[2][key] = self.group.deserialize(msk[2][key])
        print(msk)
        print(self.MSK)
        if(msk != self.MSK) :
            return {'SK' : 0}
        #根据公式所得
        x_1 = self.group.random(ZR)
        x_2 = (self.x - x_1) 
        t,u = self.group.random(ZR,2)
        K_1 = (self.g ** ( (t * x_1) / u)) *  (self.g ** ( (t * x_2) / u))
        K_3 =  u / t
        K_2 = []
        K_2s = []
        for sita in S:
            if  sita in self.AVK.keys() :
                first = t /self.AVK[sita]
                second = self.h(sita.encode()) / u
                temp = self.g ** (first * second)
                K_2s.append(self.group.serialize(temp))
                K_2.append(temp)
                
                
        K_1s = self.group.serialize(K_1)
        K_3s = self.group.serialize(K_3)
        return {'SK' : [K_1s, K_2s,K_3s] }
    def UpdateKeyGen(self,attribute):
        #对撤销属性进行更新，更新撤销列表和AVK、APK。
        attr = attribute
        new_attr = self.group.random(ZR)
        while(new_attr == self.AVK[attr]):
            new_attr = self.group.random(ZR)
        key = get_key(self.U_Mapping,attr)[0] 
        self.RL[key] =  new_attr / self.AVK[attr]
        RL_s = {}
        for sita in self.RL.keys():
            RL_s[sita] = self.group.serialize(self.RL[sita])
        self.AVK[attr] = new_attr / self.AVK[attr]
        self.APK[attr] = self.APK[attr] ** self.AVK[attr]
        self.PK[9] = self.APK
        self.MSK[2] = self.AVK
        PK_s = []
        MSK_s = []
        PK_s.append(self.PK[0])
        PK_s.append(self.PK[1])
        PK_s.append(self.PK[2])
        PK_s.append(self.group.serialize(self.PK[3]))
        PK_s.append(self.PK[4])
        PK_s.append(self.PK[5])
        PK_s.append(self.PK[6])
        PK_s.append(self.group.serialize(self.PK[7]))
        PK_s.append(self.group.serialize(self.PK[8]))
        PK_s_9 = {}
        for key in self.PK[9].keys():
                PK_s_9[key] = self.group.serialize(self.PK[9][key])
        PK_s.append(PK_s_9)
        PK_s.append(self.group.serialize(self.PK[10]))
               
        MSK_s.append(self.group.serialize(self.MSK[0]))
        MSK_s.append(self.group.serialize(self.MSK[1]))
        MSK_s_3 = {}
        for key in self.MSK[2].keys():
            MSK_s_3[key] = self.group.serialize(self.MSK[2][key])
        MSK_s.append(MSK_s_3)
        print(self.PK)
        print(self.MSK)
        print(PK_s)
        print(MSK_s)
        
        
        return {'PK' : PK_s , 'MSK' : MSK_s,'RL' : RL_s}
    def GetAllNeed(self):
        mapping = self.U_Mapping
        PK_s = []
        MSK_s = []
        PK_s.append(self.PK[0])
        PK_s.append(self.PK[1])
        PK_s.append(self.PK[2])
        PK_s.append(self.group.serialize(self.PK[3]))
        PK_s.append(self.PK[4])
        PK_s.append(self.PK[5])
        PK_s.append(self.PK[6])
        PK_s.append(self.group.serialize(self.PK[7]))
        PK_s.append(self.group.serialize(self.PK[8]))
        PK_s_9 = {}
        for key in self.PK[9].keys():
                PK_s_9[key] = self.group.serialize(self.PK[9][key])
        PK_s.append(PK_s_9)
        PK_s.append(self.group.serialize(self.PK[10]))
        MSK_s.append(self.group.serialize(self.MSK[0]))
        MSK_s.append(self.group.serialize(self.MSK[1]))
        MSK_s_3 = {}
        for key in self.MSK[2].keys():
            MSK_s_3[key] = self.group.serialize(self.MSK[2][key])
        MSK_s.append(MSK_s_3)        
        print(self.MSK)
        print(self.PK)
        print(PK_s)
        print(MSK_s)
        return {'PK':PK_s , 'MSK' : MSK_s , 'U_mapping' : mapping}
    
    def SendRL_List(self):
        RL_s = {}
        for sita in self.RL.keys():
            RL_s[sita] = self.group.serialize(self.RL[sita])  
        return RL_s
        
        
    
        
HOST = 'localhost'
PORT = 50098

if __name__ == "__main__":
    #HOST = sys.argv[1]
    #PORT = int(sys.argv[2])
    server = TA_Handle((HOST, PORT), serverReqHandler)
    print("服务器正在运行")
    server.serve_forever()

        



           
           
    
    
        
    
    
    

        
    
    

    
        
        
    
    

  
    


importing Jupyter notebook from Hash_Util.ipynb
服务器正在运行
[104808326442739892572054717159369811040598718683, 52124345308919120198671356169627777032752157366, {'podhale': 177060706100195753442959333440584847224833459966, 'accomplish': 97428564898451993304272295053036274303075780030, 'galois': 54086165783091425758578212528211274273849000655, 'bradenham': 157145479691146913241527433314727361289512009089, 'chouzenoux': 152108034014417802765155675627594021207185689285, 'pseudolaosensis': 25872953285645740912212847116324509068390005455, 'shaddad': 125522030236234623733009781322848539700818578326, 'macleandoris': 200302014059461692817205069559380275588672178673, 'vonhayat': 6004610443640719471743341868869627895195869678, 'pygmaea': 104502398337079124310608981457100282839467334144, 'blot': 87410678209185602770157302748493612412448620117, 'paraprofessionals': 44153622107504221375880582985527861121063263706, 'possibile': 132094710562976140323116689853550481966410471435, 'gyllenhal': 12217387642793

[104808326442739892572054717159369811040598718683, 52124345308919120198671356169627777032752157366, {'podhale': 177060706100195753442959333440584847224833459966, 'accomplish': 97428564898451993304272295053036274303075780030, 'galois': 54086165783091425758578212528211274273849000655, 'bradenham': 157145479691146913241527433314727361289512009089, 'chouzenoux': 152108034014417802765155675627594021207185689285, 'pseudolaosensis': 25872953285645740912212847116324509068390005455, 'shaddad': 125522030236234623733009781322848539700818578326, 'macleandoris': 200302014059461692817205069559380275588672178673, 'vonhayat': 6004610443640719471743341868869627895195869678, 'pygmaea': 104502398337079124310608981457100282839467334144, 'blot': 87410678209185602770157302748493612412448620117, 'paraprofessionals': 44153622107504221375880582985527861121063263706, 'possibile': 132094710562976140323116689853550481966410471435, 'gyllenhal': 12217387642793382021154263024085136238195574857, 'pld': 18413396633852

[104808326442739892572054717159369811040598718683, 52124345308919120198671356169627777032752157366, {'podhale': 187028898511299347052999834187625507809194108213, 'accomplish': 97428564898451993304272295053036274303075780030, 'galois': 54086165783091425758578212528211274273849000655, 'bradenham': 157145479691146913241527433314727361289512009089, 'chouzenoux': 152108034014417802765155675627594021207185689285, 'pseudolaosensis': 25872953285645740912212847116324509068390005455, 'shaddad': 125522030236234623733009781322848539700818578326, 'macleandoris': 200302014059461692817205069559380275588672178673, 'vonhayat': 6004610443640719471743341868869627895195869678, 'pygmaea': 104502398337079124310608981457100282839467334144, 'blot': 87410678209185602770157302748493612412448620117, 'paraprofessionals': 44153622107504221375880582985527861121063263706, 'possibile': 132094710562976140323116689853550481966410471435, 'gyllenhal': 12217387642793382021154263024085136238195574857, 'pld': 18413396633852

KeyboardInterrupt: 